# Simple multi agent research team

This notebook demonstrates the use of a simple multi-agent system built with the crewai library for performing a research task. The system simulates a research team with an AI Researcher, AI Expert Supervisor, and Technical Writer, who collaborate to propose, assess, and write an abstract for a research paper on recent advances in LLMs (Large Language Models)

🤖 **Simple Multi agent system as a research assistant** This is a simple multi agent system to perform a task to find a novel and impactful research contribution. In this simple example no tools and no memory are used, and simple sequential proccess is used.

By: [Ibrahim Sobh](https://www.linkedin.com/in/ibrahim-sobh-phd-8681757/)


## Install

In [1]:
%pip install -U --quiet  langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.1 MB/s eta 0:00:00


In [2]:
!pip install --quiet langchain-google-genai

In [3]:
!pip install --quiet crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.7/360.7 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 39.2 MB/s e

## LLM

In [4]:
from IPython.display import Markdown

In [5]:
# Run this cell and paste the API key in the prompt
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.5, top_p=0.85)

In [7]:
from langchain import PromptTemplate

# generate
prompt = PromptTemplate.from_template("Tell me a very short title for a facebook post about {subject}.")
gen_chain = prompt | llm

In [8]:
# Generate
generated_txt = gen_chain.invoke({"subject": "Software testing"})
generated_txt.content

'Unveiling the Secrets of Software Excellence'

## Crew


In [9]:
from crewai import Agent, Task, Process, Crew

In [10]:
Researcher = Agent(
    role="AI Researcher",
    goal="Find out the recent advances of AI in the area of LLMs",
    backstory="""You are an expert at understanding and analyzing the AI recent research papers, advances and tools. This is crucial for
		finding out a novel research contribution to attract a wide audience of students and researchers. You are good at coming up
		with new and impactful contributions to the AI research.
		""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=True,  # enable collaboration between agent
    llm=llm
)

In [12]:
Supervisor = Agent(
    role="AI Expert",
    goal="Make assessment on how contrbution is novel and impactful.",
    backstory="""You are an AI expert supervisor with a deep understanding of AI, LLMs and recent related research.
		Your expertise lies not just in knowing the technology but in foreseeing how it can be leveraged to solve real-world problems and drive business innovation.
		Your insights are crucial.""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=True,  # enable collaboration between agent
    llm=llm
)

In [13]:
Writer = Agent(
    role="Technical writre",
    goal="Write research abstract",
    backstory="""You are a technical writer expert in writing research papers in the area of AI and LLMs.
		Your expertise lies in writing professional title and abstract of a paper given the main idea or the contribution.
		The generated text must be clear and suitable to research papers.""",
    verbose=True,  # enable more detailed or extensive output
    allow_delegation=False,  # enable collaboration between agent
    llm=llm
)

In [14]:
task1 = Task(
    description="""Explore of the main research advances in the area of LLMs.
		Propose one novel and impactful idea or contribution as a new research or paper proposal.
    """,
    agent=Researcher,
    expected_output='A paragraph of the new LLMs proposed idea or contribution for a research paper.'
)

task2 = Task(
    description="""Analyze proposals for new ideas or contributions in terms of novelty and impact.
		Make a report with clear assessment.
    """,
    agent=Supervisor,
    expected_output='An assessment report with at least one clear proposed enhancment.'
)

task3 = Task(
    description="""Analyze and summarize the new research ideas or contributions and write a title and abstract of a research paper.
    The title should be professional, intersting and relevant to the area of AI and LLMs.
    The abstract has to be clear and concise with at least 7 sentences, adressing the mian contribution and expected findings.
		The generated text must be clear and suitable to research papers.
    """,
    agent=Writer,
    expected_output='Title of a paper, and the abstract'
)

In [15]:
crew = Crew(
    agents=[Researcher, Supervisor, Writer],
    tasks=[task1, task2, task3],
    verbose=2,
    process=Process.sequential,
)

result = crew.kickoff()

 [2024-08-09 11:20:24][DEBUG]: == Working Agent: AI Researcher
 [2024-08-09 11:20:24][INFO]: == Starting Task: Explore of the main research advances in the area of LLMs.
		Propose one novel and impactful idea or contribution as a new research or paper proposal.
    


> Entering new CrewAgentExecutor chain...
Action: Delegate work to coworker
Action Input: {
    'task': 'Provide me with the recent advances of AI in the area of LLMs',
    'context': 'I need to know the recent advances of AI in the area of LLMs, so I can propose a novel and impactful idea or contribution as a new research or paper proposal.',
    'coworker': 'AI Expert'
}

> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer
Final Answer: Recent advances in AI in the area of LLMs include:

**1. Improved Language Generation:** LLMs have become more proficient in generating human-like text, including creative writing, code, and translations. This has led to applications in content creation, natu

In [16]:
Markdown(result.raw)

**Title:** Leveraging LLMs for Personalized Learning in Higher Education: A Novel Framework for Enhancing Student Engagement and Success

**Abstract:**

This research paper presents a novel framework for leveraging Large Language Models (LLMs) to enhance personalized learning in higher education. The proposed framework integrates LLMs into the learning process, providing students with real-time assistance, feedback, and guidance that is tailored to their individual needs and preferences. The framework adopts a holistic approach to personalized learning by considering not only the student's academic performance but also their cognitive abilities, learning preferences, and motivational factors. The paper discusses the novelty and impact of the proposed idea, highlighting its potential to improve student engagement, increase success rates, and create a more positive and fulfilling learning experience. Additionally, the paper proposes an enhancement to the framework by exploring the use of LLMs to create personalized learning experiences for students with disabilities, making higher education more accessible and inclusive for all. The findings of this research have the potential to revolutionize the way students learn in higher education, providing them with personalized learning experiences that are tailored to their individual needs and goals.